In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import os



In [2]:

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Adjust '..' as needed
if project_root not in sys.path:
    sys.path.append(project_root)
os.chdir(project_root)

In [6]:
from sklearn.model_selection import train_test_split
df=pd.read_csv('data/data.csv')
# Features and label
X = df.drop('Bankrupt?', axis=1)
y = df['Bankrupt?']


# Split the data into training and testing sets+
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y,random_state=42)

In [7]:
from src import Custom_Transformers,data_preprocessing,Data_Resampling,model_training,model_evaluation

In [13]:
%load_ext autoreload


In [14]:

preprocess=data_preprocessing.PreprocessingPipeline()
pipeline=preprocess.preprocessing()



In [15]:
pipeline

Pipeline(steps=[('imputer', DataFrameImputer()),
                ('constant_filter', DataFrameVarianceThreshold()),
                ('outlier_capper', OutlierCapper()),
                ('cube_root_transform', DF_cube_root_transform()),
                ('scaler', DataFrameScaler()),
                ('feature_selector', FeatureSelector())])

In [16]:
%autoreload 2

In [17]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('imputer', DataFrameImputer()),
                ('constant_filter', DataFrameVarianceThreshold()),
                ('outlier_capper', OutlierCapper()),
                ('cube_root_transform', DF_cube_root_transform()),
                ('scaler', DataFrameScaler()),
                ('feature_selector', FeatureSelector())])

In [18]:
X_train_transformed=pipeline.transform(X_train)
X_test_transformed=pipeline.transform(X_test)



In [19]:
r=Data_Resampling.Resample(X_train_transformed,y_train)
X_resampled,y_resampled=r.resampling()


In [30]:
from src.model_training import ModelTrainer
trainer=ModelTrainer(X_resampled,y_resampled)


In [31]:
trainer.optimize_rf(n_trials=20)

In [ ]:
trainer.optimize_xgb(n_trials=20)

In [33]:
trainer.train_stacked_model()

In [37]:
trainer.stacked_model

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(max_depth=18,
                                                       n_estimators=155,
                                                       random_state=42)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None...
                                              interaction_constraints=None,
                                              learning_rate=0.3387294941827358,
                                              max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=200, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=42, ...))],
                   final_estimator=LogisticRegression())

In [59]:
# import joblib

# # Assume 'model' is your trained model
# model_filename = ''
# joblib.dump(trainer, model_filename)


['trained_model.bin']

In [ ]:
# import joblib

# # Load the model
# model_filename = 'trained_model.bin'
# loaded_model = joblib.load(model_filename)

# # # Use the loaded model for prediction or further evaluation
# # y_pred = loaded_model.predict(X_test)


In [51]:
from src.model_evaluation import ModelEvaluator
m=ModelEvaluator(trainer.stacked_model,X_test_transformed,y_test)
results=m.evaluate()

In [52]:
# results=m.evaluate()

In [54]:
results['confusion_matrix']

array([[1241,   79],
       [  11,   33]])

In [56]:
print(results['classification_report'])

              precision    recall  f1-score   support

           0       0.99      0.94      0.97      1320
           1       0.29      0.75      0.42        44

    accuracy                           0.93      1364
   macro avg       0.64      0.85      0.69      1364
weighted avg       0.97      0.93      0.95      1364

